# Single step predictor

This notebook shows how to load the predictor, make some predictions, and postprocess them.

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from mlgan.generator.run import run as build_generator
from mlgan.core import preprocessing, postprocessing, data_sources



C:\Users\nitha\Anaconda3\envs\AIArch\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [1]:
from pathlib import Path
import random
import logging

import numpy as np
import cv2
import ast
import matplotlib.pyplot as plt

In [6]:
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(module)s - %(message)s"
)

In [7]:
input_images = Path('../data/images/complete_floorplan/input/')

In [8]:
config_generator = data_sources.load_yaml(Path("../config/generator/complete_floorplan.yaml"))
config_generator['settings']['input_shape'] = ast.literal_eval(config_generator['settings']['input_shape'])

2023-06-03 19:18:30,346 - INFO - yaml_files - Loading yaml file from ..\config\generator\complete_floorplan.yaml.


In [11]:
generator = build_generator(config_generator)

2023-06-03 19:19:45,619 - INFO - BaseGenerator - Loading model for category: models.
2023-06-03 19:19:45,651 - WARNING - normalizations - The given value for groups will be overwritten.
2023-06-03 19:19:45,651 - WARNING - normalizations - The given value for groups will be overwritten.
2023-06-03 19:19:45,651 - WARNING - normalizations - The given value for groups will be overwritten.
2023-06-03 19:19:45,666 - WARNING - normalizations - The given value for groups will be overwritten.
2023-06-03 19:19:45,666 - WARNING - normalizations - The given value for groups will be overwritten.
2023-06-03 19:19:45,682 - WARNING - normalizations - The given value for groups will be overwritten.
2023-06-03 19:19:45,687 - WARNING - normalizations - The given value for groups will be overwritten.
2023-06-03 19:19:45,687 - WARNING - normalizations - The given value for groups will be overwritten.
2023-06-03 19:19:45,698 - WARNING - normalizations - The given value for groups will be overwritten.
2023-0

In [12]:
input_images_list = [x for x in input_images.glob('*.png')]
len(input_images_list)

1764

In [ ]:
random_file = random.choice(input_images_list)

input_image = cv2.imread(str(random_file))
input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)

predictions = generator.predict(input_image, 5)
generator.visualize_predictions(input_image, predictions=predictions);

1/1 [==============================] - 0s 266ms/step


In [ ]:
postprocessed_predictions = generator.postprocess_predictions(predictions=predictions, input_img=input_image)

In [ ]:
preprocessed_img = preprocessing.resize_and_pad(
    input_image, size=(256, 256), pad_color=255
)

in_img_gdf = preprocessing.image_to_geodataframe(preprocessed_img)
in_img_gdf.geometry = [in_img_gdf.unary_union] * in_img_gdf.shape[0]
in_img_gdf = in_img_gdf.head(1).explode().reset_index(drop=True)

In [ ]:
n_predictions = len(postprocessed_predictions.keys())
fig, axes = plt.subplots(nrows=int(np.ceil(n_predictions/2)), ncols=2, figsize=(30, 30))
axes = axes.flatten()
for _preds, ax in zip(postprocessed_predictions.items(), axes):
    in_img_gdf.exterior.buffer(1).plot(ax=ax, color='black')

    _key, _res = _preds
    _res.plot(color=_res['colors'], ax=ax)
    _res.exterior.plot(color='black', ax=ax)
    ax.set_title(_key)

for ax in axes:
    ax.axis('off')
fig.suptitle('Postprocessed floorplans', fontsize=16, y=1)
plt.tight_layout();